<a href="https://colab.research.google.com/github/ATIK2110018/Earth_engine/blob/main/teesta_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
!pip install geemap

In [18]:
import ee
import geemap

ee.Authenticate()
ee.Initialize(project='atik-2110018')
Map = geemap.Map()

In [19]:
shapefile_path = '/content/drive/MyDrive/GIS data/teesta_project/aoi/aoi.shp'

In [20]:
!pip install pycrs

In [21]:
aoi = geemap.shp_to_ee(shapefile_path)

In [22]:
image = (
    ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")
    .filterBounds(aoi)
    .filterDate("1990-10-01", "1991-10-30")
    .filter(ee.Filter.lt('CLOUD_COVER', 1))
    .sort("CLOUD_COVER")
    .mosaic()
    .select('SR_B[1-7]')
)

vis_params = {"min": 0, "max": 25000, "bands": ["SR_B4", "SR_B3", "SR_B2"]}

Map.centerObject(aoi, 8)
Map.addLayer(image, vis_params, "Landsat-5")
Map.addLayer(aoi, {}, "AOI")

In [23]:
land_1990= image.clip(aoi)
Map.addLayer(land_1990, vis_params, "land_1990")
Map

Map(center=[26.054918001644495, 89.11593911157134], controls=(WidgetControl(options=['position', 'transparent_…

In [24]:
props = image.propertyNames()

print(image.get("CLOUD_COVER").getInfo())

None


In [25]:
training = land_1990.sample(
    region=aoi.geometry(), # Provide the geometry of the aoi as the region
    scale=30,
    numPixels=10000,
    seed=0,
    geometries=True
)

Map.addLayer(training, {}, "training", False)
Map

Map(center=[26.054918001644495, 89.11593911157134], controls=(WidgetControl(options=['position', 'transparent_…

In [26]:
n_clusters = 3
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

In [27]:
result = land_1990.cluster(clusterer)

Map.addLayer(result.randomVisualizer(), {}, "clusters")

In [28]:
class_names = ['Bar', 'Vegetation', 'Water']
class_palette = {i: name for i, name in enumerate(class_names)}

color_dict = {
    'blue': (0, 0, 255),
    'green': (0, 128, 0),
    'red': (255, 0, 0)
}

class_colors = ['red', 'green', 'blue']

legend_dict = {class_names[i]: color_dict[class_colors[i]] for i in range(len(class_names))}

classified_image = result.remap(
    ee.List(list(class_palette.keys())),
    ee.List(list(class_palette.keys())),
    0
).rename('land_cover')

Map.addLayer(
    classified_image,
    {'min': 0, 'max': 2, 'palette': class_colors},
    'Classified Image'
)

Map.add_legend(legend_dict=legend_dict, title="Land Cover Classification")

Map

Map(center=[26.054918001644495, 89.11593911157134], controls=(WidgetControl(options=['position', 'transparent_…

In [29]:
original_values = [0, 1, 2]
target_values = [2, 4, 6]

classified_image_remapped = classified_image.remap(
    ee.List(original_values),
    ee.List(target_values),
    0
).rename('land_cover')

In [30]:
# geemap.ee_export_image(
#     classified_image_remapped,
#     filename='classified_image.tif',
#     scale=30,
#     region=aoi.geometry(),
#     file_per_band=False
# )

Generating URL ...
Please wait ...
Data downloaded to /content/classified_image.tif
